In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Selected device:", device)

torch.manual_seed(42)

In [ ]:
class RNNCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, bias = True):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.bias = bias

        self.Wi = nn.Linear(input_dim + hidden_dim, hidden_dim, bias = bias)
        
    def forward(self, inputs, hidden):
        h = hidden
        concat_ih = torch.cat((inputs,h), 1)

        h = self.Wi(concat_ih)
        
        return h

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, bias = True):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.Layer = RNNCell(input_dim = embedding_dim, hidden_dim = hidden_dim, bias = bias)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, inputs):

        batch_size = inputs.size(0)
        seq_length = inputs.size(1)

        x = self.embedding(inputs)

        h = torch.zeros(batch_size, self.hidden_dim)

        RNN = self.Layer
        for t in range(seq_length):
            input_t = x[:, t, :]
            h = RNN.forward(input_t, h)

        output = F.sigmoid(self.fc(h))

        return output

In [ ]:
class LSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, bias = True):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.bias = bias

        self.Wi = nn.Linear(input_dim + hidden_dim, hidden_dim, bias = bias)
        self.Wf = nn.Linear(input_dim + hidden_dim, hidden_dim, bias = bias)
        self.Wo = nn.Linear(input_dim + hidden_dim, hidden_dim, bias = bias)
        self.Wh = nn.Linear(input_dim + hidden_dim, hidden_dim, bias = bias)
        self.W
        
    def forward(self, input, hidden):
        h, c = hidden
        concat_ih = torch.cat((input, h), 1)

        input_gate = F.sigmoid(self.Wi(concat_ih))
        forget_gate = F.sigmoid(self.Wf(concat_ih))
        output_gate = F.sigmoid(self.Wo(concat_ih))
        cell_gate = F.tanh(self.Wh(concat_ih))

        c = forget_gate * c + input_gate * cell_gate
        h = output_gate * F.tanh(c)

        return h, c

In [ ]:
class DeepLSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, output_dim, bias = True):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.Layers = nn.ModuleList([LSTMCell(input_dim = embedding_dim, hidden_dim = hidden_dim, bias = bias)])
        for i in range(num_layers - 1):
            self.Layers.append(LSTMCell(input_dim = hidden_dim, hidden_dim = hidden_dim, bias = bias))

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, inputs):

        batch_size = inputs.size(0)
        seq_length = inputs.size(1)

        x = self.embedding(inputs)
        
        for i in range(len(self.Layers)):

            LSTM = self.Layers[i]
            
            h = torch.zeros(batch_size, self.hidden_dim)
            c = torch.zeros(batch_size, self.hidden_dim)
            hidden = (h, c)

            for t in range(seq_length):

                input_t = x[:, t, :]
                hidden = LSTM.forward(input_t, hidden)
                x[:, t, :] = hidden[0]

        output = F.sigmoid(self.fc(h))

        return output